In [9]:
TEMPLATE = """
cl run
    :semeval07 :coinco :swords :code :orenmel_lexsub
    "
    mkdir -p /swords/assets/parsed;
    mkdir -p /swords/assets/public/raw/data/semeval07;
    mkdir -p /swords/assets/public/raw/eval;
    cp -r coinco/* /swords/assets/parsed/;
    cp -r swords/* /swords/assets/parsed/;
    cp -r code/* /swords/swords/;
    cp semeval07 /swords/assets/public/raw/data/semeval07/task10data.tar.gz;
    cp orenmel_lexsub/lexsub-master/jcs/evaluation/measures/generalized_average_precision.py /swords/assets/public/raw/eval/;
    python -m swords.cli run {dataset} --generator {generator} --ranker {ranker} --output_result_json_fp {dataset}_{generator}_{ranker}.lsr.json;
    python -m swords.cli eval {dataset} --result_json_fp {dataset}_{generator}_{ranker}.lsr.json --output_metrics_json_fp metrics.json --metrics {metrics} | tee eval_stdout.txt;
    tail -2 eval_stdout.txt > metrics.txt
    "
    -n Table{tab}.{model}
    --request-memory 30g
    --request-docker-image chrisdonahue/swords
    --request-network
    --request-gpus {num_gpus}
"""

SWORDS_SUBSET_NAME = 'swords-v1.0_test-subset'
SWORDS_TEST_NAME = 'swords-v1.0_test'


for tab, model, dataset, generator, ranker, metrics in [
    # Table 4
    (4, 'Humans', SWORDS_SUBSET_NAME, 'swords-reannotated-test-acceptable', 'identity', 'lenient_a_f@10'),
    (4, 'Humans', SWORDS_SUBSET_NAME, 'swords-reannotated-test-conceivable', 'identity', 'lenient_c_f@10'),
    (4, 'CoInCo', SWORDS_TEST_NAME, 'coinco-test', 'identity', 'lenient_a_f@10,lenient_c_f@10'),
    (4, 'Thesaurus', SWORDS_TEST_NAME, 'thesaurus', 'identity', 'lenient_a_f@10,lenient_c_f@10'),
    (4, 'Thesaurus_Ranked', SWORDS_TEST_NAME, 'thesaurus', 'bert-contextual', 'lenient_a_f@10,lenient_c_f@10'),
    (4, 'BERT-M', SWORDS_TEST_NAME, 'bert-based-ls-bert-mask', 'identity', None),
    (4, 'BERT-M_Ranked', SWORDS_TEST_NAME, 'bert-based-ls-bert-mask', 'bert-contextual', None),
    (4, 'BERT-K', SWORDS_TEST_NAME, 'bert-based-ls-bert-keep', 'identity', None),
    (4, 'BERT-K_Ranked', SWORDS_TEST_NAME, 'bert-based-ls-bert-keep', 'bert-contextual', None),
    (4, 'BERT-LS', SWORDS_TEST_NAME, 'bert-based-ls', 'bert-based-ls', None),
    (4, 'GPT-3', SWORDS_TEST_NAME, 'gpt3-swords-v0.8-test', 'identity', None),
    (4, 'GPT-3_Ranked', SWORDS_TEST_NAME, 'gpt3-swords-v0.8-test', 'bert-contextual', None),
    (4, 'WordTune', SWORDS_TEST_NAME, 'wordtune-swords-v0.8-test', 'identity', None),
    (4, 'Wordtune_Ranked', SWORDS_TEST_NAME, 'wordtune-swords-v0.8-test', 'bert-contextual', None),
    # Table 6
    (6, 'Humans', SWORDS_SUBSET_NAME, 'swords-reannotated-test-all', 'identity', 'gap_rat'),
    (6, 'BERT', SWORDS_TEST_NAME, 'oracle-plus-distractors', 'bert-contextual', 'gap_rat'),
    (6, 'BERT-LS_no_sp', SWORDS_TEST_NAME, 'oracle-plus-distractors', 'bert-based-ls-wosp', 'gap_rat'),
    (6, 'BERT-LS', SWORDS_TEST_NAME, 'oracle-plus-distractors', 'bert-based-ls', 'gap_rat'),
    (6, 'BERT-LS_no_sv', SWORDS_TEST_NAME, 'oracle-plus-distractors', 'bert-based-ls-wosv', 'gap_rat'),
    (6, 'GloVe', SWORDS_TEST_NAME, 'oracle-plus-distractors', 'glove', 'gap_rat'),
    (6, 'Random', SWORDS_TEST_NAME, 'oracle-plus-distractors', 'nonsense', 'gap_rat'),
    # Other
    (8, 'Oracle', SWORDS_SUBSET_NAME, 'oracle-conceivable', 'identity', 'strict_c_p@1'),
]:
    assert len(model.split()) == 1
    if metrics is None:
        metrics = 'lenient_a_f@10,lenient_c_f@10,strict_a_f@10,strict_c_f@10'
    num_gpus = 1 if 'bert' in generator or 'bert' in ranker else 0
    
    cmd = TEMPLATE.format(
        dataset=dataset,
        generator=generator,
        ranker=ranker,
        metrics=metrics,
        num_gpus=num_gpus,
        tab=tab,
        model=model)
    print(' '.join([l.strip() for l in cmd.strip().splitlines()]))
    print()

cl run :semeval07 :coinco :swords :code :orenmel_lexsub " mkdir -p /swords/assets/parsed; mkdir -p /swords/assets/public/raw/data/semeval07; mkdir -p /swords/assets/public/raw/eval; cp -r coinco/* /swords/assets/parsed/; cp -r swords/* /swords/assets/parsed/; cp -r code/* /swords/swords/; cp semeval07 /swords/assets/public/raw/data/semeval07/task10data.tar.gz; cp orenmel_lexsub/lexsub-master/jcs/evaluation/measures/generalized_average_precision.py /swords/assets/public/raw/eval/; python -m swords.cli run swords-v1.0_test-subset --generator swords-reannotated-test-acceptable --ranker identity --output_result_json_fp swords-v1.0_test-subset_swords-reannotated-test-acceptable_identity.lsr.json; python -m swords.cli eval swords-v1.0_test-subset --result_json_fp swords-v1.0_test-subset_swords-reannotated-test-acceptable_identity.lsr.json --output_metrics_json_fp metrics.json --metrics lenient_a_f@10 | tee eval_stdout.txt; tail -2 eval_stdout.txt > metrics.txt " -n Table4.Humans --request-me